In [ ]:
include("BarrierSynthesis.jl")

# 0. Init system variable and parameters

In [ ]:
#init state and input variables
@polyvar u[1:2]
@polyvar(x[1:7]) # x[1] is x, x[2] is y, x[3] is vel_x, x[4] is vel_y, x[5] is θ, x[6] is ω, x[7] is error

#define system dynamics
dynamics = [
        (ctrl -> [x[3],x[4], (ctrl[1]+ctrl[2])*(-2.0/π)*(x[5]+π)-0.2x[7], (ctrl[1]+ctrl[2])*((2.0/π)*(x[5] + π/2.0)-0.2x[7])-2, x[6], ctrl[1]-ctrl[2], 0.]),
        (ctrl -> [x[3],x[4], (ctrl[1]+ctrl[2])*(2.0/π)*x[5]+0.2x[7], (ctrl[1]+ctrl[2])*((2.0/π)*(x[5] + π/2.0)-0.2x[7])-2, x[6], ctrl[1]-ctrl[2], 0.]),
        (ctrl -> [x[3],x[4], (ctrl[1]+ctrl[2])*(2.0/π)*x[5]+0.2x[7], (ctrl[1]+ctrl[2])*((-2.0/π)*(x[5] - π/2.0)+0.2x[7])-2, x[6], ctrl[1]-ctrl[2], 0.]),
        (ctrl -> [x[3],x[4], (ctrl[1]+ctrl[2])*(-2.0/π)*(x[5]-π)-0.2x[7], (ctrl[1]+ctrl[2])*((-2.0/π)*(x[5] - π/2.0)+0.2x[7])-2, x[6], ctrl[1]-ctrl[2], 0.]),
    ]

#define unsafe set (obstacle)
g = 0.1^2 - x[1]^2 - x[2]^2 # Obstacle: is a (x,y) ball of radius 0.1 around origin

#state and input bounds
bounds = [[-10., 10.], [-10., 10.], [-10.,10.], [-10.,10.],[-π,π], [-10.,10.],[-1.,1.]]

In [ ]:
vector_field = [
        ([x[3],x[4], (u[1]+u[2])*(-2.0/π)*(x[5]+π)-0.2x[7], (u[1]+u[2])*((2.0/π)*(x[5] + π/2.0)-0.2x[7])-2, x[6], u[1]-u[2], 0.]),
        ([x[3],x[4], (u[1]+u[2])*(2.0/π)*x[5]+0.2x[7], (u[1]+u[2])*((2.0/π)*(x[5] + π/2.0)-0.2x[7])-2, x[6], u[1]-u[2], 0.]),
        ([x[3],x[4], (u[1]+u[2])*(2.0/π)*x[5]+0.2x[7], (u[1]+u[2])*((-2.0/π)*(x[5] - π/2.0)+0.2x[7])-2, x[6], u[1]-u[2], 0.]),
        ([x[3],x[4], (u[1]+u[2])*(-2.0/π)*(x[5]-π)-0.2x[7], (u[1]+u[2])*((-2.0/π)*(x[5] - π/2.0)+0.2x[7])-2, x[6], u[1]-u[2], 0.]),
]

In [ ]:
#instantiate parameters
λ = 1
ϵ = 1
max_degree=4
ul = 2.
U = [[0., 0.], [0., ul], [ul, 0.], [ul, ul]]
n_tests = 400
u_bounds = [[0, ul], [0, ul]]

In [ ]:
# generate test points
test_pts = [ get_random(bounds, g) for _ in 1:n_tests];

# 1. Computing initial set of barriers for each input U_i

In [ ]:
# function to comupte transit time for each barrier 

function refine_barrier(x, u, bounds, u_bounds, g, vectorField, B, B_dot; ϵ = 0.25, κ = 50.)
    function prepare_domain_hybrid(var, lb, ub)
        dom = @set(var >= lb) ∩ @set(var <= ub) ∩ @set( (var-lb)*(ub-var) >= 0)
        return dom
    end
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
#     println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ >=0)
#     @variable(model, 0 <= B <= κ)
#     @variable(model, B_dot)
    
#     B_dot = dot(differentiate(B,x), vectorField)
    dom3 = dom ∩ @set(B >= 0) ∩ @set(B <= κ)
    for (j,B_dot_j) in enumerate(B_dot)
        dom_j = prepare_domain_hybrid(x[4], (j-3) * π/2, (j-2) * π/2)
        @constraint(model, -η*B - δ <= B_dot_j, domain=dom3 ∩ dom_j)
    end
        @constraint(model, η*κ + δ >= 0)

    #set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(value(η))")
    println("δ = $(value(δ))")
    τ =  (κ)/max(value(δ),value(η) *  κ + value(δ))
    println("τd = $(value(τ))")
    return τ
end

In [ ]:
elapsed1 = @elapsed begin
B_1, B_1d, lm1 = findBarrierFixedControlInput_HybridPM(x,U[1],g,dynamics,vector_field,test_pts)
end
display(B_1)
t_1 = refine_barrier(x, u, bounds, u_bounds, g, dynamics, B_1,  B_1d)
test_pts = filter!(pt -> B_1(pt...) <= 0., test_pts)

In [ ]:
elapsed2 = @elapsed begin
B_2, B_2d, lm2 = findBarrierFixedControlInput_HybridPM(x,U[2],g,dynamics,vector_field,test_pts)
end
display(B_2)
# display(B_2d)
t_2 = refine_barrier(x, u, bounds, u_bounds, g, dynamics, B_2,  B_2d)
test_pts = filter!(pt -> B_2(pt...) <= 0., test_pts)

In [ ]:
elapsed3 = @elapsed begin
B_3, B_3d, lm3 = findBarrierFixedControlInput_HybridPM(x,U[3],g,dynamics,vector_field,test_pts)
end
display(B_3)
# display(B_3d)
t_3 = refine_barrier(x, u, bounds, u_bounds, g, dynamics, B_3,  B_3d)
test_pts = filter!(pt -> B_3(pt...) <= 0., test_pts)

In [ ]:
test_pts = [ get_random(bounds, g) for _ in 1:50];
elapsed4 = @elapsed begin
B_4, B_4d, lm4 = findBarrierFixedControlInput_HybridPM(x,U[4],g,dynamics,vector_field,test_pts)
end
display(B_4)
# display(B_4d)
t_4 = refine_barrier(x, u, bounds, u_bounds, g, dynamics, B_4,  B_4d)
test_pts = filter!(pt -> B_4(pt...) <= 0., test_pts)

# 2. Computing Successive Barriers

In [ ]:
# functions to compute successive barriers

In [ ]:
function refine_barrier_succ(x, u, bounds, u_bounds, g, vectorField, B, B_dot, ancestors; ϵ = 0.25, κ = 50.)
    function prepare_domain_hybrid(var, lb, ub)
        dom = @set(var >= lb) ∩ @set(var <= ub) ∩ @set( (var-lb)*(ub-var) >= 0)
        return dom
    end
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
#     println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ >=0)
#     @variable(model, 0 <= B <= κ)
#     @variable(model, B_dot)
    
#     B_dot = dot(differentiate(B,x), vectorField)
    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(b <= 0) for b in ancestors]))
    else
        new_domain = dom 
    end
    
    dom3 = dom ∩ @set(B >= 0) ∩ @set(B <= κ)
    for (j,B_dot_j) in enumerate(B_dot)
        dom_j = prepare_domain_hybrid(x[4], (j-3) * π/2, (j-2) * π/2)
        @constraint(model, -η*B - δ <= B_dot_j, domain=dom3 ∩ dom_j)
    end
        @constraint(model, η*κ + δ >= 0)

    #set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(value(η))")
    println("δ = $(value(δ))")
    τ =  (κ)/max(value(δ),value(η) *  κ + value(δ))
    println("τd = $(value(τ))")
    return τ
end

In [ ]:
ancestors = [B_1, B_2, B_3, B_4]
s_elapsed = @elapsed begin
(second_level_barriers, test_pts_1) = compute_next_level_barriersPM(x, u, bounds, g, dynamics, vector_field, U, test_pts, ancestors)
end

In [ ]:
all_barriers = [B for (B,_) in second_level_barriers];
first_level = [B_1, B_2, B_3, B_4];

In [ ]:
test_pts_1 = [ get_random(bounds, g) for _ in 1:25];
ancestors2 = [ancestors; all_barriers];
(third_level_barriers, test_pts_2) = compute_next_level_barriersPM(x, u, bounds, g, dynamics, vector_field, U, test_pts, ancestors)

# 3. Benchmarks

In [ ]:
# generating data for Table 1 in the paper (row for current system)

In [ ]:
println("Time taken for B1: $(elapsed1+elapsed2+elapsed3+elapsed4)")
println("# barriers B1: $(size(first_level))")
println("Time taken for B2: $(s_elapsed)")
println("# barriers B2: $(size(all_barriers))")

# 5. Verification

In [ ]:
"""
Certifying barriers using constraint PSD check
"""

using DelimitedFiles

function check_psd_constraints(multipliers_array::Vector)
    counter = 0
    M = []
    for mults in multipliers_array
        for lms in mults
            for lm in lms
                if !isposdef(Matrix(lm.Q)) && !isposdef(Diagonal(svd(Matrix(lm.Q)).S))
                    counter += 1
                else 
#                     display(Matrix(lm.Q))
                    push!(M, [Matrix(lm.Q)])
                end
            end
        end
    end
    open("matrices_pm.txt", "w") do io
        writedlm(io, M)
    end
    if counter==0
        println("All constraints are PSD: Barrier certified")
    else
        println("PSD check failed!")
    end
end

In [ ]:
all_barrier_plus = push!([second_level_barriers], third_level_barriers);

In [ ]:
lms=[]
for i in all_barrier_plus
    if (!ismissing(i[1][4]))
        lms = append!(lms,i[1][4])
    end
end

In [ ]:
check_psd_constraints([lm1,lm2, lm3, lm4, lms])